In [37]:
import pandas as pd
import os
pd.set_option('display.float_format', '{:.5f}'.format)

# Metrics taken in account

## Accuracy

Accuracy is calculated as:

$$
\text{Accuracy} = \frac{\text{True Positive + True Negative}}{\text{True Positive + False Positive + True Negative + False Negative}}
$$


## Precision

The fraction of produced detections which are true positives.

$$
\text{Precision} = \frac{\text{True Positive}}{\text{True Positive + False Positive}} = \frac{\text{True Positive}}{\text{Total Number of Prediction}}
$$


## Recall

The fraction of groundtruth boxes in the data that matched to some produced detection.

$$
\text{Recall} = \frac{\text{True Positive}}{\text{True Positive + False Negative}}
$$

## F1 score

F1 calcula el balance entre precision y recall. Si el F1 es alto, precision y recall son altos.

The F1 Score is calculated as:

$$
\text{F1 Score} = \frac{2 \times (\text{Precision} \times \text{Recall})}{\text{Precision} + \text{Recall}}
$$

# The problem

We are dealing with a weapon detection issue. It is more important to avoid false negatives than false positives. It is better not to miss a weapon detection, at the cost of detecting more weapons with less accuracy. For this reason, a low threshold will also be used.

Given the context of weapon detection and the emphasis on avoiding false negatives (missing weapon detections) at the expense of potentially having more false positives (incorrect weapon detections), **recall** is more important than precision in this scenario.

Recall, also known as sensitivity or true positive rate, measures the proportion of actual positive cases (weapons) that were correctly identified by the model. In the context of your problem, a higher recall means that the model is effectively capturing a larger portion of actual weapons, minimizing the risk of missing any potentially dangerous objects.

While precision (positive predictive value) is also significant, it prioritizes the accuracy of positive predictions among all predicted positives. In your case, a focus on precision might lead to being overly cautious and producing fewer false positives, but it could also result in missing actual weapon detections, which is a more critical concern in a weapon detection scenario.

A higher **recall** rate is crucial for ensuring that potential weapons are not overlooked, even if it means accepting a higher number of false positives.

# Evaluating models vs small dataset
## Training 50 epochs

In [38]:
OUTPUT_DIR = "./output_csvs"
for filename in os.listdir(OUTPUT_DIR):
    if filename.endswith(".csv"):
        print(filename)

eval_trained_on_dataset_v1_task_test_ds_original.csv
eval_trained_on_dataset_v1_task_val_ds_original.csv
eval_trained_on_dataset_v2_task_test_ds_original.csv
eval_trained_on_dataset_v1_task_test_ds_rc.csv
eval_trained_on_dataset_v2_task_test_ds_rc.csv
eval_trained_on_dataset_v2_task_val_ds_original.csv


In [40]:
df = pd.read_csv(f"{OUTPUT_DIR}/eval_trained_on_dataset_v1_task_test_ds_original.csv")
# df.set_index('model_key', inplace=True)

columns_of_interest = ['model','imgsz','transfer_learning', 'lr0', 'optimizer', 'all_P', 'all_R',
                       'all_F1', 'all_mAP@.5', 'all_mAP@.5:.95']

df[(df['epochs'] == 50)][columns_of_interest].sort_values(by=['all_P'])

,model,imgsz,transfer_learning,lr0,optimizer,all_P,all_R,all_F1,all_mAP@.5,all_mAP@.5:.95
1,yolov5,800,no,0.01000,SGD,0.73700,0.62500,0.33820,0.70100,0.35800
4,yolov5,640,yolov5s,0.01000,SGD,0.77300,0.71200,0.37062,0.76400,0.42900
6,yolov5,800,yolov5s,0.01000,SGD,0.79600,0.71600,0.37694,0.75500,0.42800
9,yolov7,640,yolov7training,0.00100,SGD,0.83900,0.90200,0.43468,0.90700,0.62600
12,yolov8,800,no,0.01000,SGD,0.84200,0.74400,0.39499,0.84100,0.57900
18,yolov8,800,yolov8s,0.01000,SGD,0.86400,0.77600,0.40882,0.85200,0.58300
16,yolov8,640,yolov8s,0.00100,Adam,0.86800,0.78200,0.41138,0.85700,0.59500


### Initial conclusions

The experiments were on yolov5, yolov7 and yolov8, and are evaluated on different image sizes, number of epochs, transfer learning and learning rate. The first experiments were trained in 50 epochs. 

Among the models, YOLOv8 generally performs better than YOLOv5 and YOLOv7 in terms of precision, but YOLOv7 performs better in recall and in mAP.

#### Transfer learning
Some experiments use transfer learning by starting with pre-trained weights ('yolov5s.pt', 'yolov7_training.pt', 'yolov8s.pt'). Transfer learning seems to generally improve performance in terms of precision, recall, and F1-score.

#### Loss function and learning rate

A smaller learning rate leads to more optimal models but takes longer to train. The ADAM optimizer seems to have better accuracy than the SGD optimizer.

## Training 100 epochs 

In [41]:
df[(df['epochs'] == 100)][columns_of_interest].sort_values(by=['all_P'])

,model,imgsz,transfer_learning,lr0,optimizer,all_P,all_R,all_F1,all_mAP@.5,all_mAP@.5:.95
2,yolov5,640,yolov5s,0.00100,SGD,0.69200,0.60300,0.32222,0.66500,0.33300
0,yolov5,800,no,0.01000,SGD,0.79300,0.68900,0.36868,0.74700,0.42200
3,yolov5,640,yolov5s,0.01000,SGD,0.81100,0.75000,0.38965,0.80700,0.47600
14,yolov8,640,yolov8s,0.00100,Adam,0.85200,0.79600,0.41152,0.84900,0.61600
5,yolov5,800,yolov5s,0.01000,SGD,0.85400,0.74500,0.39789,0.80700,0.49000
10,yolov7,800,yolov7training,0.01000,SGD,0.85600,0.80800,0.41565,0.85800,0.58300
13,yolov8,800,yolov8l,0.01000,SGD,0.86700,0.75800,0.40442,0.83800,0.59700
7,yolov7,640,yolov7training,0.00100,SGD,0.87000,0.90400,0.44334,0.91400,0.64000
11,yolov8,800,no,0.01000,SGD,0.87100,0.77600,0.41038,0.84600,0.59900
8,yolov7,640,yolov7training,0.01000,SGD,0.87400,0.84900,0.43066,0.88100,0.62400


Increasing the number of training epochs from 50 to 100 seems to have improved the performance of most models across various metrics.

Higher epochs might allow the models to learn more complex patterns, leading to better results.

From the table, we can draw the following conclusions:

1. **Size Doesn't Always Mean Better:** The table suggests that having a bigger model does not necessarily guarantee better performance. For instance, the yolov8l model, which is larger than yolov8s, does not perform better.

2. **Impact of Image Size:** For every model, it is shown that having a 640 imgsz gives a better recall.

3. **Model Version:** Different versions of YOLO show differences in performance. Some versions might perform better in terms of certain metrics. In terms of recall, yolov7 performs better than yolov5 and yolov8. In terms of precision, the best model was achieved with yolov8.

4. **Transfer learning:** Across various configurations, using transfer learning consistently yields higher precision (P), recall (R), F1-score (F1), and mean Average Precision (mAP) metrics compared to non-transfer learning. This highlights the effectiveness of leveraging pre-trained models for better object detection performance.

5. **Precision and Recall Trade-off:** It's clear that there's a trade-off between precision and recall. Some models have higher precision but lower recall, while others have higher recall but lower precision.

Besides the metrics, the practical aspects like computational resources and inference speed will be considered choosing a model for deployment.

In [51]:
df = pd.read_csv(f"{OUTPUT_DIR}/eval_trained_on_dataset_v1_task_test_ds_rc.csv")

df[columns_of_interest].sort_values(by=['all_P'])

,model,epochs,imgsz,transfer_learning,lr0,optimizer,all_P,all_R,all_F1,all_mAP@.5,all_mAP@.5:.95
5,yolov8,50,640,yolov8s,0.00100,Adam,0.12400,0.15500,0.06889,0.06650,0.02570
1,yolov8,50,800,no,0.01000,SGD,0.13500,0.12900,0.06597,0.05420,0.02020
0,yolov8,100,800,no,0.01000,SGD,0.13700,0.11700,0.06311,0.04880,0.01980
2,yolov8,100,800,yolov8l,0.01000,SGD,0.15700,0.12700,0.07021,0.06150,0.02610
4,yolov8,100,640,yolov8s,0.01000,SGD,0.15900,0.10700,0.06396,0.04840,0.01990
7,yolov8,50,800,yolov8s,0.01000,SGD,0.21600,0.17600,0.09698,0.08590,0.02960
6,yolov8,100,800,yolov8s,0.01000,SGD,0.22200,0.11800,0.07705,0.07260,0.03070
3,yolov8,100,640,yolov8s,0.00100,Adam,0.32300,0.11000,0.08206,0.09760,0.03340


# Training models on a bigger dataset

Ideally, considering the previous evaluations, the optimal choice for training models on a larger dataset would have been yolov7. However, due to resource limitations on the training platform (Google Colab), specifically in terms of memory allocation, it proved unfeasible. Attempts to train the models on a bigger dataset resulted in memory overflows, abruptly terminating the training process before the model could be effectively trained... So, the next option was yolov8.

- image size: 640
- batch size: 16
- transfer learning: yolov8s
- lr: between 0.001 (with Adam) and 0.01 (with SGD)
- epochs: 100 and 300, to test if training longer gives better results

In [47]:
df = pd.read_csv(f"{OUTPUT_DIR}/eval_trained_on_dataset_v2_task_test_ds_original.csv")

columns_of_interest = ['model','epochs','imgsz','transfer_learning', 'lr0', 'optimizer', 'all_P', 'all_R',
                       'all_F1', 'all_mAP@.5', 'all_mAP@.5:.95']

df[columns_of_interest].sort_values(by=['all_P'])

,model,epochs,imgsz,transfer_learning,lr0,optimizer,all_P,all_R,all_F1,all_mAP@.5,all_mAP@.5:.95
4,yolov8,100,640,yolov8s,0.00100,Adam,0.98600,0.96200,0.48693,0.98100,0.94100
7,yolov8,100,800,yolov8s,0.01000,SGD,0.98700,0.96100,0.48691,0.98200,0.94900
10,yolov8,80,800,yolov8s,0.01000,SGD,0.98700,0.96100,0.48691,0.98200,0.94800
3,yolov8,50,640,yolov8m,0.01000,SGD,0.99100,0.96200,0.48814,0.97400,0.93800
2,yolov8,100,640,yolov8m,0.01000,SGD,0.99300,0.96200,0.48863,0.97700,0.94100
9,yolov8,60,800,yolov8s,0.01000,SGD,0.99300,0.96200,0.48863,0.97700,0.94600
6,yolov8,50,640,yolov8s,0.00100,Adam,0.99700,0.95500,0.48777,0.97900,0.93200
8,yolov8,40,800,yolov8s,0.01000,SGD,0.99700,0.92400,0.47956,0.98000,0.93200
0,yolov8,50,640,yolov8l,0.00100,Adam,1.00000,0.94100,0.48480,0.97200,0.93200
1,yolov8,50,640,yolov8l,0.01000,SGD,1.00000,0.95900,0.48954,0.98400,0.93700


In [54]:
df = pd.read_csv(f"{OUTPUT_DIR}/eval_trained_on_dataset_v2_task_test_ds_rc.csv")

df[columns_of_interest].sort_values(by=['all_P'])

,model,epochs,imgsz,transfer_learning,lr0,optimizer,all_P,all_R,all_F1,all_mAP@.5,all_mAP@.5:.95
6,yolov8,50,640,yolov8s,0.00100,Adam,0.30700,0.30200,0.15224,0.21400,0.08270
3,yolov8,50,640,yolov8m,0.01000,SGD,0.33100,0.34500,0.16893,0.28500,0.11400
0,yolov8,50,640,yolov8l,0.00100,Adam,0.35200,0.27600,0.15470,0.21000,0.07960
8,yolov8,40,800,yolov8s,0.01000,SGD,0.37200,0.32700,0.17403,0.26800,0.11000
4,yolov8,100,640,yolov8s,0.00100,Adam,0.43500,0.28600,0.17255,0.26900,0.10200
1,yolov8,50,640,yolov8l,0.01000,SGD,0.48100,0.34700,0.20158,0.30900,0.13000
5,yolov8,300,640,yolov8s,0.01000,SGD,0.50900,0.31100,0.19305,0.32500,0.13700
10,yolov8,80,800,yolov8s,0.01000,SGD,0.56000,0.30200,0.19619,0.31000,0.12300
2,yolov8,100,640,yolov8m,0.01000,SGD,0.57600,0.33500,0.21181,0.36600,0.14500
7,yolov8,100,800,yolov8s,0.01000,SGD,0.57800,0.30200,0.19836,0.31400,0.12400


me gustan los ultimos 3 (a revisar)

In [55]:
df[columns_of_interest].sort_values(by=['all_R'])

,model,epochs,imgsz,transfer_learning,lr0,optimizer,all_P,all_R,all_F1,all_mAP@.5,all_mAP@.5:.95
0,yolov8,50,640,yolov8l,0.00100,Adam,0.35200,0.27600,0.15470,0.21000,0.07960
4,yolov8,100,640,yolov8s,0.00100,Adam,0.43500,0.28600,0.17255,0.26900,0.10200
9,yolov8,60,800,yolov8s,0.01000,SGD,0.58100,0.29300,0.19477,0.31800,0.12500
6,yolov8,50,640,yolov8s,0.00100,Adam,0.30700,0.30200,0.15224,0.21400,0.08270
7,yolov8,100,800,yolov8s,0.01000,SGD,0.57800,0.30200,0.19836,0.31400,0.12400
10,yolov8,80,800,yolov8s,0.01000,SGD,0.56000,0.30200,0.19619,0.31000,0.12300
5,yolov8,300,640,yolov8s,0.01000,SGD,0.50900,0.31100,0.19305,0.32500,0.13700
8,yolov8,40,800,yolov8s,0.01000,SGD,0.37200,0.32700,0.17403,0.26800,0.11000
2,yolov8,100,640,yolov8m,0.01000,SGD,0.57600,0.33500,0.21181,0.36600,0.14500
3,yolov8,50,640,yolov8m,0.01000,SGD,0.33100,0.34500,0.16893,0.28500,0.11400


In [ ]:
el 2